In [223]:
import string
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np
from keras.utils import to_categorical
from keras import Sequential
from keras.layers import Dense, LSTM, Embedding, Dropout
import pickle
import pandas as pd

In [5]:
def load_data(filename='all_songs.csv',
              col='album',
              col_value='In the Aeroplane Over the Sea'):
    
    df    = pd.read_csv('all_songs.csv')
    df    = df.loc[df[col] == col_value]
    songs = df['lyrics'].values
    return songs

In [6]:
def lyric_cleaner(songs):
    lyric_tokens = []
    for song in songs:
        text = song.lower().replace(' n ', ' eol ').replace('[verse ', '[verse')
        text = text.replace("'", '').replace('-', ' ')
        tokens = text.split()
        table = str.maketrans('', '', string.punctuation)
        tokens = [word.translate(table) for word in tokens]
        lyric_tokens.append(tokens)
    
    return lyric_tokens

In [8]:
def join_song_lyrics(lyric_tokens):
    
    joined_songs = []
    
    for song in lyric_tokens:
        lyrics = ' '.join(song)
        joined_songs.append(lyrics)
        
    return joined_songs

In [9]:
def tokenizer(list_of_songs, tokenizer=Tokenizer()):
    tokenizer = tokenizer
    tokenizer.fit_on_texts(list_of_songs)
    encoded_songs = tokenizer.texts_to_sequences(list_of_songs)
    
    return encoded_songs, tokenizer

In [10]:
def find_longest_song(encoded_songs):
    longest_song_size = 0
    for song in encoded_songs:
        if len(song) > longest_song_size:
            longest_song_size = len(song)
    return longest_song_size

In [11]:
def index_to_word(tokenizer):
    vocab_size = len(tokenizer.word_index) + 1
    index_to_word_dict =  dict([(index, word) for word, index in tokenizer.word_index.items()])
    return vocab_size, index_to_word_dict

In [13]:
def pad_data(encoded_songs, max_length):
    padded_songs = pad_sequences(encoded_songs, maxlen=max_length, padding='post', truncating='post')
    return padded_songs

In [44]:
def seqeuncer(padded_songs, length=seq_length):
    sequences = []
    output = []

    for song in padded_songs:
        for i in range(length, (len(song))):
            seq = song[i - length: i]
            out = song[i]
            sequences.append(seq)
            output.append(out)
            
    return np.array(sequences), output

In [231]:
def establish_model(vocab_size=vocab_size, seq_length=seq_length):
    model = Sequential()
    model.add(Embedding(vocab_size, 50, input_length=seq_length))
    model.add(LSTM(100, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(100, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(100, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(vocab_size, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [132]:
def gen_seed(seqs=sequences):
    seed_text = seqs[np.random.randint(0,len(seqs))]
    while seed_text.all() == np.array([  0, 0,  0, 0, 0, 0, 0, 0, 0, 0]).all():
        seed_text = seqs[np.random.randint(0,len(seqs))]
    return seed_text

In [237]:
def generate_seq(model,
                 tokenizer,
                 seq_length,
                 seqs=sequences,
                 n_words=1000):
    
    seed_text = gen_seed()
    start = [reverse_dict[index] for index in seed_text]
    print('start: ', start)
    
    result = []
    
    for _ in range(n_words):
        encoded = pad_sequences([seed_text], maxlen=seq_length, truncating='pre')
        yhat = model.predict_classes(encoded, verbose=0)
        if yhat == 0:
            pass
        else:
            word = reverse_dict[yhat[0]]
        seed_text = np.append(seed_text, yhat)
        seed_text = seed_text[-10:]
        result.append(word)
    return ' '.join(result)

In [203]:
seq_length = 8

data = load_data()

data = data[:11]

lyric_tokens = lyric_cleaner(data)

list_of_songs = join_song_lyrics(lyric_tokens)

encoded_songs, toke = tokenizer(list_of_songs=list_of_songs)

vocab_size, reverse_dict = index_to_word(tokenizer=toke)

longest_song_size = find_longest_song(encoded_songs=encoded_songs)

padded_songs = pad_data(encoded_songs, max_length=longest_song_size)

X, y = seqeuncer(padded_songs=padded_songs, length=seq_length)

X = np.array(X)

y = to_categorical(y)

In [204]:
nmh_model = establish_model(seq_length=8)

nmh_model.fit(X, y, batch_size=128, epochs=100)

Epoch 1/100
5973/5973 [==============================] - 6s 1ms/step - loss: 4.1911 - acc: 0.5707
Epoch 2/100
5973/5973 [==============================] - 4s 674us/step - loss: 2.3816 - acc: 0.5826
Epoch 3/100
5973/5973 [==============================] - 5s 798us/step - loss: 2.2883 - acc: 0.5826
Epoch 4/100
5973/5973 [==============================] - 4s 706us/step - loss: 2.2340 - acc: 0.6134
Epoch 5/100
5973/5973 [==============================] - 4s 682us/step - loss: 2.1952 - acc: 0.6322
Epoch 6/100
5973/5973 [==============================] - 4s 643us/step - loss: 2.1571 - acc: 0.6352
Epoch 7/100
5973/5973 [==============================] - 4s 641us/step - loss: 2.1119 - acc: 0.6387
Epoch 8/100
5973/5973 [==============================] - 3s 440us/step - loss: 2.0739 - acc: 0.6362
Epoch 9/100
5973/5973 [==============================] - 4s 656us/step - loss: 2.0306 - acc: 0.6374
Epoch 10/100
5973/5973 [==============================] - 4s 629us/step - loss: 1.9985 - acc: 0.6412
E

Epoch 82/100
5973/5973 [==============================] - 4s 752us/step - loss: 0.4014 - acc: 0.8939
Epoch 83/100
5973/5973 [==============================] - 5s 799us/step - loss: 0.3854 - acc: 0.8975
Epoch 84/100
5973/5973 [==============================] - 5s 757us/step - loss: 0.3745 - acc: 0.9032
Epoch 85/100
5973/5973 [==============================] - 4s 708us/step - loss: 0.3617 - acc: 0.9081
Epoch 86/100
5973/5973 [==============================] - 4s 653us/step - loss: 0.3525 - acc: 0.9096
Epoch 87/100
5973/5973 [==============================] - 4s 654us/step - loss: 0.3346 - acc: 0.9150
Epoch 88/100
5973/5973 [==============================] - 4s 636us/step - loss: 0.3158 - acc: 0.9208
Epoch 89/100
5973/5973 [==============================] - 5s 761us/step - loss: 0.3096 - acc: 0.9205
Epoch 90/100
5973/5973 [==============================] - 4s 628us/step - loss: 0.3022 - acc: 0.9248
Epoch 91/100
5973/5973 [==============================] - 4s 602us/step - loss: 0.2925 - ac

In [222]:
# save model
nmh_model.save('nmh_seq_len_8_batch_size_128_epochs_100_padded.h5')

In [220]:
generate_seq(model=nmh_model, tokenizer=toke, seq_length=seq_length)

start:  ['her', 'side', 'eol', 'and', 'only', 'weeks', 'before', 'the', 'guns', 'eol']


'that came world others the miracle eol afraid that she a eol will on all your licking we sweet eol when room eol dark eol you and this all this heart off exist brother twist on im you at machine verse2 eol mean and eol of what hed on that you the eol and a cloud an eol sounding mother that we you you in in doorway need eol need the sweet find find secret found me love i eol eol eol but all curious we friends your time me dee dee dee dee dee eol eol shit goldaline my dear all we comforting weeks dad in eol eol memory right 1945 right memory her sister with pretty bright drunk didnt eol eol that away the find asleep with lose eol eol she drink little every and so warm white in eol cloud agree only and roses some one eyes leave de eol mean i everyone bends eol one we live and now never in would shes eol to with longer pistons were spit wanting drenched drenched when fleshy wait eol oh be buried til here cry love exist that struggle she please so side rest make life the learn eol verse2 c

In [221]:
generate_seq(model=nmh_model, tokenizer=toke, seq_length=seq_length)

start:  ['over', 'the', 'sea', 'eol', 'but', 'for', 'now', 'we', 'are', 'young']


'eol let us lay eol the sun eol and and every and live green forever eol your mouth to now eol can for the as we to mean curious a all all we put you it proclaiming bubbly that keep inside eol to water to things with eol when ear eol headed eol boy some im night thing me eol eol two verse3 dont and eol only and them them and that eol needle where was still that when mine secrets eol keep needed you eyes you in the letting there miracles on the the three eol chorus eol your he sun your your for undertow eol the and would brother your when you weed until up to you the with with eol everything the who you me eol eol eol now just and where i chest dog sing in in more dad and to shoulder anything to learn eol to guess eol with eol in all exist brother her only only standing them it it in still have that well tap chorus while her in she moves little doesnt ill will with eol now move we your your swim eol later will eol than fell across again eol and on eol eol she all the eol that was with w

In [225]:
# with dropout

In [232]:
nmh_model = establish_model(seq_length=8)

nmh_model.fit(X, y, batch_size=128, epochs=100)

Epoch 1/100
5973/5973 [==============================] - 7s 1ms/step - loss: 4.0521 - acc: 0.5665
Epoch 2/100
5973/5973 [==============================] - 4s 713us/step - loss: 2.3534 - acc: 0.5826
Epoch 3/100
5973/5973 [==============================] - 4s 715us/step - loss: 2.2720 - acc: 0.5830
Epoch 4/100
5973/5973 [==============================] - 5s 830us/step - loss: 2.2369 - acc: 0.6099
Epoch 5/100
5973/5973 [==============================] - 5s 773us/step - loss: 2.2004 - acc: 0.6325
Epoch 6/100
5973/5973 [==============================] - 5s 770us/step - loss: 2.1750 - acc: 0.6339
Epoch 7/100
5973/5973 [==============================] - 4s 662us/step - loss: 2.1472 - acc: 0.6355
Epoch 8/100
5973/5973 [==============================] - 4s 666us/step - loss: 2.1246 - acc: 0.6372
Epoch 9/100
5973/5973 [==============================] - 5s 840us/step - loss: 2.0988 - acc: 0.6374
Epoch 10/100
5973/5973 [==============================] - 4s 747us/step - loss: 2.0653 - acc: 0.6375
E

In [238]:
generate_seq(model=nmh_model, tokenizer=toke, seq_length=seq_length)

start:  ['dissolve', 'and', 'drain', 'away', 'eol', 'the', 'world', 'it', 'goes', 'eol']


'and always waits eol dark on round the sun eol her and i until voice time again eol soft now day and on a break how lay aimed you on on do eol verse3 eol up and and is found flowers the guess eol and lay there there on eol rest how swim eol bends music music the with watching eye away eol here spine eol on and and little across fist eol in i will girl sparks eol need glass eol signals ghost that sun eol what you pulleys to choking with with to with her at mine for me eol i was life your have we silly it is fly eol music music world had eol and choking sweetness hands loved loved bright across eol so out around many that i is within boy still eol that silly keep when keep carrot young eol ring black eol we hate pick enough every eol two for moving eol each dream keep thought i have eol jesus weed water tumbling from the knew eol i love like pull be out away eol standing know with they thing know want shoes eol creating eol creek fork about machine her two spine eol moving eol wave coul

In [239]:
generate_seq(model=nmh_model, tokenizer=toke, seq_length=seq_length)

start:  ['breaking', 'eol', 'everything', 'that', 'you', 'could', 'keep', 'inside', 'eol', 'now']


'your eyes aint moving eol now they just lay there in their climb eol verse3 eol two headed boy eol there on child flame eol the world it every we carrot for ringing eol over eol her will a all you white no shoes to make to when hear when muscles here eol ovaries world sheets eol bursting say 14 once bubbly enough me eol but alive their machine bright all days eol let it listening to want shoes eol it it to played and bright accordion terrible border eol left only were fleshy sunday shoes eol same eol the sounds to sparks pistons pistons wanting aeroplane when life stains stains to me eol and sound the i lazy the that dogs dissolve eol with with where anything to dead be go eol but comely we their faces pick carrot others eol that now bring playing playing head know eol on on and on eol world on eol and mouth all in grieve eol proves place place place the marriage is all your heres i hear love eol i am day and love eol how others miracle eol and mouth is to he played holy you thundercl

In [240]:
nmh_model.save('nmh_seq_len_8_batch_size_128_epochs_100_padded_two_dense100_two_lstm100_3dropout2.h5')